In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
import pandas as pd
import numpy as np
from openbt.openbt import OPENBT
from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from copy import deepcopy
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn.gaussian_process import GaussianProcessRegressor
# import econml
# from sklearn.base import BaseEstimator


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv",
    names = ["treatment", "y_factual", "y_cfactual", "mu0", "mu1"] + [f'x{x}' for x in range(25)])

X = df.loc[:,"x0":].to_numpy()
t = df["treatment"].to_numpy()
#t = np.zeros_like(t)
y = df["y_factual"].to_numpy()
y1 = np.where(df["treatment"] == 1,
               df['y_factual'],
               df['y_cfactual'])
y0 = np.where(df["treatment"] == 0,
               df['y_factual'],
               df['y_cfactual'])
ite = np.where(df["treatment"] == 1,
               df['y_factual'] - df["y_cfactual"],
               df['y_cfactual'] - df["y_factual"])

In [ ]:
X_train, X_test, t_train, t_test, y_train, y_test, ite_train, ite_test, y1_train, y1_test, y0_train, y0_test = train_test_split(
    X, t, y, ite, y1, y0,  test_size=0.9, random_state=1005)
ds = {"X_training": X_train,
     "y_training": y_train,
     "t_training": t_train,
     "X_pool": deepcopy(X_test), 
     "y_pool": deepcopy(y_test),
     "t_pool": deepcopy(t_test),
     "y1_pool": y1_test,
     "y0_pool":y0_test,
     "X_test": X_test,
     "y_test": y_test,
      "t_test": t_test,
      "ite_test": ite_test
     }
asl = BaseActiveLearner(estimator = BaseITEEstimator(model = RandomForestRegressor(),
                                         two_model=False,ps_model=None),
                        acquisition_function=RandomAcquisitionFunction(),
                        assignment_function=MajorityAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)

In [ ]:
asl.fit()
X_new, query_idx = asl.query(no_query=120)
asl.teach(query_idx)
preds = asl.predict(asl.dataset["X_test"])
asl.score()

In [ ]:
bite = OPENBTITEEstimator(model=OPENBT(model="bart"),
                          two_model = False,
                          ps_model  = None,
                          dataset   = ds)
bite.fit()
preds_obt = bite.predict(X = ds["X_test"])
#print(preds_obt.shape)

Overwriting k to agree with the model's default
Overwriting overallnu to agree with the model's default
Overwriting ntree to agree with the model's default
Overwriting ntreeh to agree with the model's default
Overwriting overallsd to agree with the model's default
Writing config file and data
/var/folders/44/gtm_t6x110jg6b13p4rbwkfh0000gn/T/openbtpy_61ha1j47
3+ x variables
Running model...


In [ ]:
bite = CausalForestEstimator(
                        two_model = None,
                        ps_model  = None,
                        dataset   = ds)
bite.fit()
preds_cf = bite.predict(X = ds["X_test"])

In [ ]:
cite = CEVAEEstimator(dataset=ds,two_model=None)
cite.fit()
preds_ce = cite.predict(X=ds["X_test"])

INFO 	 Training with 1 minibatches per epoch
DEBUG 	 step     0 loss = 51.8192
INFO 	 Evaluating 7 minibatches
DEBUG 	 batch ate = 1.44438
DEBUG 	 batch ate = 1.41592
DEBUG 	 batch ate = 1.41316
DEBUG 	 batch ate = 1.52459
DEBUG 	 batch ate = 1.4511
DEBUG 	 batch ate = 1.43276
DEBUG 	 batch ate = 1.68415


In [ ]:
gite = GPEstimator(model=GaussianProcessRegressor(),
                          two_model = True,
                          ps_model  = None,
                          dataset   = ds)
gite.fit()
preds_gp = gite.predict(X = ds["X_test"])

In [ ]:
# blr = BLREstimator(two_model=False, ps_model = None, dataset=ds)
# 
# blr.fit()
# 
# blr.predict(X=ds["X_test"])